Copyright **`(c)`** 2024 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

# Set Cover problem

See: https://en.wikipedia.org/wiki/Set_cover_problem

In [1]:
from random import random, seed
from itertools import product,accumulate
import numpy as np
from tqdm.auto import tqdm
from icecream import ic
from matplotlib import pyplot as plt

## Reproducible Initialization

If you want to get reproducible results, use `rng` (and restart the kernel); for non-reproducible ones, use `np.random`.

In [337]:
UNIVERSE_SIZE = 100000
NUM_SETS = 10000
DENSITY = 0.1
MAX_STEPS = 10_000

rng = np.random.Generator(np.random.PCG64([UNIVERSE_SIZE, NUM_SETS, int(10_000 * DENSITY)]))



In [338]:
# DON'T EDIT THESE LINES!

SETS = np.random.random((NUM_SETS, UNIVERSE_SIZE)) < DENSITY
for s in range(UNIVERSE_SIZE):#this fot is done in order to make sure that each element is in at least one set
    if not np.any(SETS[:, s]):
        SETS[np.random.randint(NUM_SETS), s] = True
COSTS = np.pow(SETS.sum(axis=1), 1.1)



## Helper Functions

In [339]:
def valid(solution):
    """Checks wether solution is valid (ie. covers all universe)"""
    return np.all(np.logical_or.reduce(SETS[solution]))


def cost(solution):
    """Returns the cost of a solution (to be minimized)"""
    return COSTS[solution].sum()

def fitness(solution):
    return (valid(solution),-cost(solution))

## Have Fun!

In [ ]:
# A dumb solution of "all" sets
solution = np.full(NUM_SETS, True)
valid(solution), cost(solution)


In [ ]:

solution = rng.random(NUM_SETS) < 1
valid(solution), cost(solution)

## MULTI-MUTATION Tweak 
# parameters: solution that we want to mutate, p = probability of multi-mutation

In [342]:

def tweak_multi_mutation(solution,p):
    new_solution = solution.copy()
    index = None
    while index is None or np.random.random() < p:
        index = rng.integers(0, NUM_SETS)
        new_solution[index] = not new_solution[index]
    return new_solution



In [343]:
tweak=tweak_multi_mutation

In [ ]:
history = [fitness(solution)[1]]
ic(cost(solution))
for _ in tqdm(range(MAX_STEPS)):
    new_solution= tweak(solution,0.2)
    f=fitness(new_solution)
    history.append(f[1])
    if f > fitness(solution):
        solution = new_solution

ic(cost(solution))
ic(history.index(-cost(solution)))

plt.figure(figsize=(14, 8))
plt.plot(
    range(len(history)),
    list(accumulate(history, max)),
    color="red",
)
_ = plt.scatter(range(len(history)), history, marker=".")
plt.savefig(f"Multi-Mutation {UNIVERSE_SIZE}-{NUM_SETS}-{DENSITY}.png")


## Simulated Annealing

In [332]:
tweak=tweak_multi_mutation
solution = rng.random(NUM_SETS) < 1

In [ ]:
history = [fitness(solution)[1]]
ic(cost(solution))
for step in tqdm(range(MAX_STEPS)):
    new_solution= tweak(solution,0.2)
    f=fitness(new_solution)
    history.append(f[1])
    if f > fitness(solution):
        solution = new_solution
    else:
        temperture = 1-step/MAX_STEPS
        #pick only valid solutions
        if f[0] and np.random.random() < np.exp((f[1]-fitness(solution)[1])/temperture):
            ic("pick worsen solution")
            solution = new_solution
    
ic(-cost(solution))
ic(history.index(-cost(solution)))

plt.figure(figsize=(14, 8))
plt.plot(
    range(len(history)),
    list(accumulate(history, max)),
    color="red",
)
_ = plt.scatter(range(len(history)), history, marker=".")
plt.savefig(f"Simulated Annealing {UNIVERSE_SIZE}-{NUM_SETS}-{DENSITY}.png")


## Steepest - Step hill climbing

In [334]:
tweak=tweak_multi_mutation
solution = rng.random(NUM_SETS) < 1

In [ ]:
NUM_RESTARTS = 3
STEEPEST_STEP_CANDIDATES = 5
max_value = fitness(solution)
num_steps = 0
history = list()
last_improvement = (0, 0)
for i in tqdm(range(NUM_RESTARTS), position=0, desc="Restarts"):
    #big change in the solution (p large)
    solution = tweak(solution,0.7)
    while not valid(solution):
        solution = tweak(solution,0.7)
    f = fitness(solution)
    history.append(f[1])
    new_solution = solution
    for n in tqdm(range(1000), position=1):
        # create candidate solutions with exploitation (p small)
        candidates = [tweak(solution,0.1) for i in range(0, STEEPEST_STEP_CANDIDATES)]
        candidates_fitness = list()
        for c in candidates:
            f = fitness(c)
            history.append(f[1])
            candidates_fitness.append(f)
        idx = candidates_fitness.index(max(candidates_fitness))

        new_solution = candidates[idx]
        new_fitness = candidates_fitness[idx]
        num_steps += STEEPEST_STEP_CANDIDATES

        if new_fitness > fitness(solution):
            last_improvement = (i, n)
            solution = new_solution

    if fitness(solution) > max_value:
        max_value = fitness(solution)
        best_solution = solution

print(f"\nBest solution: {max_value} found at {last_improvement} -- total steps: {num_steps}")
ic(cost(solution))
ic(history.index(-cost(solution)))

plt.figure(figsize=(14, 8))
plt.plot(
    range(len(history)),
    list(accumulate(history, max)),
    color="red",
)

_ = plt.scatter(range(len(history)), history, marker=".")

plt.savefig(f"Steepest-step {UNIVERSE_SIZE}-{NUM_SETS}-{DENSITY}.png")